# This notebook is for me to explore the data
Im just going to look at how offen co ocurences happen and dont happen (to see if a sparse matrix is a good idea), also I want to look at the total number of items for an aditional catigorisation step to see if i can do some clever PCA (SVD) to beter deal with verry cardinal purchacing by customers.

I decided to include this in the repo to show my working not to try and pick up marks, as such please ignore my poor spelling.

In [33]:
from itertools import combinations
from collections import Counter
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from scipy import sparse

In [12]:
dataset = pd.read_csv("data/Supermarket_dataset_PAI.csv")

In [17]:
dataset

,itemDescription,Purchace_ID
0,tropical fruit,180820150721
1,whole milk,255220150105
2,pip fruit,230020150919
3,other vegetables,118720151212
4,whole milk,303720150201
...,...,...
38760,sliced cheese,447120141008
38761,candy,202220140223
38762,cake bar,109720140416
38763,fruit/vegetable juice,151020141203


In [13]:
date_str = pd.to_datetime(dataset["Date"], format="%d-%m-%Y").dt.strftime("%Y%m%d")
dataset["Purchace_ID"] = (dataset["Member_number"].astype(str) + date_str).astype(int)
dataset = dataset.drop(columns=["Member_number", "Date"])
dataset

,itemDescription,Purchace_ID
0,tropical fruit,180820150721
1,whole milk,255220150105
2,pip fruit,230020150919
3,other vegetables,118720151212
4,whole milk,303720150201
...,...,...
38760,sliced cheese,447120141008
38761,candy,202220140223
38762,cake bar,109720140416
38763,fruit/vegetable juice,151020141203


In [16]:
print(dataset["Purchace_ID"].nunique())
print(dataset["itemDescription"].nunique())

14963
167


In [ ]:
# can this be a uint32?
dataset["Purchace_ID"].max() < (2**32 - 1)
# no :( but i dont think I acutualy need to do that

500020150210

In [ ]:
# Map each item to an index
items = np.sort(dataset["itemDescription"].unique())
item_idx = {item: i for i, item in enumerate(items)}

# Count co-purchases per Purchace_ID
pair_counts = Counter()
for _, grp in dataset.groupby("Purchace_ID"):
    uniq_items = np.unique(grp["itemDescription"])
    # off-diagonal co-purchases
    for a, b in combinations(uniq_items, 2):
        pair_counts[(a, b)] += 1
        pair_counts[(b, a)] += 1
    # diagonal (item with itself)
    for item in uniq_items:
        pair_counts[(item, item)] += 1

# Build sparse matrix (rows/cols are items)
rows, cols, data = zip(
    *((item_idx[a], item_idx[b], count) for (a, b), count in pair_counts.items())
)
co_matrix = coo_matrix((data, (rows, cols)), shape=(len(items), len(items))).tocsr()

co_matrix

<167x167 sparse matrix of type '<class 'numpy.int64'>'
	with 12687 stored elements in Compressed Sparse Row format>

In [36]:
co_matrix.shape

(167, 167)

In [37]:
co_matrix.size

12687

so basicly it bearly makes sense to use a sparse matrix, but a: it does make sense (>1/2 of elements != 0)and b: It makes much more sense in any store with a resonably sized selection.

wallmart ~ 120k items <br>
tesco ~ 50k <br>
aldi ~ 2K <br>
**example = 167**
